In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load data
file_path = "Data/restaurant_preferences.xlsx"
data = pd.read_excel(file_path)

# Extract usernames and preferences
usernames = data.iloc[:, 0]  # First column is username
restaurants = data.iloc[:, 1:]  # Remaining columns are restaurant preferences

# Handle missing values by replacing with an empty string
restaurants = restaurants.fillna("")

# Convert rows of restaurant preferences to a list of unique values per user
restaurant_lists = restaurants.apply(lambda x: [r for r in x if r != ""], axis=1)

# One-hot encode the restaurant preferences
mlb = MultiLabelBinarizer()
one_hot_restaurants = mlb.fit_transform(restaurant_lists)

# Convert to a DataFrame for easier manipulation
restaurant_df = pd.DataFrame(one_hot_restaurants, columns=mlb.classes_)

# Create training data using leave-one-out for each user
X, y = [], []
for row in restaurant_df.itertuples(index=False):
    liked_restaurants = np.where(row)[0]  # Get indices of liked restaurants
    if len(liked_restaurants) < 2:
        # Skip users with fewer than 2 preferences, as we need at least one for training
        continue
    
    # Leave-one-out: for each restaurant in the liked list, create a separate training instance
    for left_out_idx in liked_restaurants:
        # Input features: All but the left-out restaurant
        input_features = np.array(row, dtype=int)
        input_features[left_out_idx] = 0  # "Leave out" one restaurant
        
        X.append(input_features)
        y.append(left_out_idx)  # Target: the index of the left-out restaurant

X, y = np.array(X), np.array(y)

# Define the neural network model
model = Sequential([
    Dense(16, activation='relu', input_shape=(X.shape[1],)),
    Dense(8, activation='relu'),
    Dense(X.shape[1], activation='softmax')  # Output layer with softmax for probabilities
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Function for predicting a left-out restaurant for a new user
def recommend_restaurant(selected_restaurants):
    # Convert selected restaurants to one-hot encoding
    input_vector = np.zeros(X.shape[1])
    for restaurant in selected_restaurants:
        if restaurant in mlb.classes_:
            input_vector[mlb.classes_.tolist().index(restaurant)] = 1
    
    # Predict the probabilities for each restaurant
    prediction = model.predict(input_vector.reshape(1, -1))
    recommended_index = np.argmax(prediction)
    return mlb.classes_[recommended_index]

Epoch 1/10
8/8 [==============================] - 1s 32ms/step - loss: 3.6188 - accuracy: 0.0292 - val_loss: 3.6181 - val_accuracy: 0.0167
Epoch 2/10
8/8 [==============================] - 0s 6ms/step - loss: 3.6074 - accuracy: 0.0292 - val_loss: 3.6096 - val_accuracy: 0.0167
Epoch 3/10
8/8 [==============================] - 0s 6ms/step - loss: 3.5974 - accuracy: 0.0292 - val_loss: 3.6015 - val_accuracy: 0.0333
Epoch 4/10
8/8 [==============================] - 0s 6ms/step - loss: 3.5878 - accuracy: 0.0375 - val_loss: 3.5942 - val_accuracy: 0.0167
Epoch 5/10
8/8 [==============================] - 0s 6ms/step - loss: 3.5792 - accuracy: 0.0500 - val_loss: 3.5862 - val_accuracy: 0.0167
Epoch 6/10
8/8 [==============================] - 0s 5ms/step - loss: 3.5698 - accuracy: 0.0542 - val_loss: 3.5794 - val_accuracy: 0.0333
Epoch 7/10
8/8 [==============================] - 0s 6ms/step - loss: 3.5605 - accuracy: 0.0542 - val_loss: 3.5729 - val_accuracy: 0.0333
Epoch 8/10
8/8 [=================

In [1]:
import tkinter as tk
from tkinter import messagebox
from tensorflow.keras.models import load_model  # If you saved your model, you can load it

# Initialize main GUI window
root = tk.Tk()
root.title("Restaurant Recommender System")
root.geometry("400x300")

# Load the unique restaurant list for dropdown options
restaurant_list = mlb.classes_.tolist()

# Variables to store selected restaurants
selected_restaurants = [tk.StringVar() for _ in range(4)]

# Create dropdowns for restaurant selection
dropdown_labels = ["Restaurant 1", "Restaurant 2", "Restaurant 3", "Restaurant 4"]
for i in range(4):
    tk.Label(root, text=dropdown_labels[i]).pack()
    tk.OptionMenu(root, selected_restaurants[i], *restaurant_list).pack()

# Function to get recommendations based on selected restaurants
def get_recommendation():
    # Gather selected restaurant choices
    selected = [var.get() for var in selected_restaurants]
    
    # Check if all dropdowns have a valid selection
    if "" in selected:
        messagebox.showwarning("Incomplete Selection", "Please select four restaurants.")
        return
    
    # Use the model to predict the recommended restaurant
    recommended_restaurant = recommend_restaurant(selected)
    messagebox.showinfo("Recommendation", f"We recommend you try: {recommended_restaurant}")

# Button to get recommendations
tk.Button(root, text="Get Recommendation", command=get_recommendation).pack()

# Run the GUI application
root.mainloop()

NameError: name 'mlb' is not defined